In [17]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import urllib.request as rq
import json
import os
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
def fetch_data(index,type):
    response = rq.urlopen(os.environ["ES_URL"]+ index+'/'+type+'/_search?size=1000000')
    data = json.load(response)   
    data = data["hits"]["hits"]
    df = json_normalize(data)
    return df

## US immigration data

In [3]:
df = fetch_data("immigration","country_yr")

In [4]:
print(df.shape)
print(df.columns)

(1520, 10)
Index(['_id', '_index', '_score', '_source.country', '_source.latitude',
       '_source.longitude', '_source.region', '_source.value', '_source.yr',
       '_type'],
      dtype='object')


### Looking at individual rows

In [7]:
df[5:10]

,_id,_index,_score,_source.country,_source.latitude,_source.longitude,_source.region,_source.value,_source.yr,_type
5,AVr7U8BIu8C0brA7OwO3,immigration,1.0,hungary,47.162494,19.503304,europe,5598,1880,country_yr
6,AVr7U8BIu8C0brA7OwO8,immigration,1.0,hungary,47.162494,19.503304,europe,29499,1930,country_yr
7,AVr7U8BOu8C0brA7OwPD,immigration,1.0,hungary,47.162494,19.503304,europe,9295,2000,country_yr
8,AVr7U8BOu8C0brA7OwPI,immigration,1.0,belgium,50.503887,4.469936,europe,3996,1850,country_yr
9,AVr7U8BOu8C0brA7OwPN,immigration,1.0,belgium,50.503887,4.469936,europe,19642,1900,country_yr


### Finding variation in total immigration by year

In [14]:
imm_by_yr = df.groupby(["_source.yr"]).sum()["_source.value"]
imm_by_yr = imm_by_yr/1000

In [15]:
imm_by_yr.describe(percentiles=[0.6,0.8,0.9,0.99])

count       20.000000
mean      3940.516800
std       2890.938527
min        109.329000
50%       3415.660500
60%       4266.558200
80%       6020.556800
90%       7621.898800
99%      10023.274300
max      10087.500000
Name: _source.value, dtype: float64

In [18]:
df["_source.value"].describe()

count      1520.000
mean      51848.905
std      163401.866
min           0.000
25%           1.000
50%        3065.000
75%       29248.000
max     2757418.000
Name: _source.value, dtype: float64

## For each region what is the variation of total immigrants

In [27]:
df[df["_source.value"]>29248].groupby(["_source.region"]).sum()["_source.value"]/1000

_source.region
africa             1578.607
america           13154.607
asia              13187.536
caribbean          4973.977
central america    1599.485
europe            33125.328
oceania              32.728
russia             3971.006
south america      2375.120
Name: _source.value, dtype: float64